# Analyzing and tuning a transmon qubit
We will showcase two methods (EPR amd LOM) to analyze the same design. Specifically, we will use here the `advanced` methods to run the simulations and analysis, which directly contorl renderers and external packages. Please refer to the tutorial notebooks 4.1 and 4.2 to follow the `suggested` flow to run the analysis.

## Index
#### Transmon design
1. Prepare the single transmon qubit layout in qiskit-metal. <br>

#### Transmon analysis using EPR method
1. Set-up and run a finite element simulate to extract the eigenmode. <br>
1. Display EM fields to inspect quality of the setup. <br>
1. Identify junction parameters for the EPR analysis. <br>
1. Run EPR analysis on single eigenmode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>

#### Transmon analysis using LOM method
1. Calculate the capacitance matrix. <br>
1. Execute analysis on extracted LOM. <br>

## Prerequisite
You need to have a working local installation of Ansys.<br>
Also you will need the following directives and inports.

In [2]:
%reload_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

# 1. Create the Qbit design

Fix the design dimensions that you intend to reflect in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [3]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

Create a single transmon in the center of the chip previously defined.

In [4]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



# 2. Analyze the transmon using the Eigenmode-EPR method

In this section we will use a semi-manual (advanced) analysis flow. Please refer to tutorial 4.2 for the `suggested` method. As illustrated, the methods are equivalent, but the advanced method allows you to directly override some renderer-specific settings.

### Finite Element Eigenmode Analysis

#### Setup

Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation.

In [9]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

For the Eigenmode simulation portion, you can either:
1. Use the `eig_qb` user-friendly methods (see tutorial 4.2)
2. Control directly the simulation tool from the tool's GUI (outside metal - see specific vendor instructions)
3. Use the renderer methods
In this section we show the advanced method (method 3).

The renderer can be reached from the analysis class. Let's give it a shorter alias.

In [10]:
hfss = eig_qb.sim.renderer

Now we connect to the tool using the unified command.

In [11]:
hfss.start()

INFO 04:54PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:54PM [load_ansys_project]: 	Opened Ansys App
INFO 04:54PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 04:54PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Berkelium/Documents/Ansoft/
	Project:   Project5
INFO 04:54PM [connect_design]: No active design found (or error getting active design).
INFO 04:54PM [connect]: 	 Connected to project "Project5". No design detected


True

The previous command is supposed to open ansys (if closed), create a new project and finally connect this notebook to it.

If for any reason the previous cell failed, please try the manual path described in the next three cells:
1. uncomment and execute only **one** of the lines in the first cell.
1. uncomment and execute the second cell.
1. uncomment and execute only **one** of the lines in the third cell.

In [ ]:
# hfss.open_ansys()   # this opens Ansys 2021 R2 if present
# hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# hfss.open_ansys(path='C:\\Program Files\\AnsysEM\\AnsysEM21.1\\Win64')
# hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM21.1/Win64'

In [ ]:
# hfss.new_ansys_project()

In [ ]:
# hfss.connect_ansys()
# hfss.connect_ansys('C:\\project_path\\', 'Project1')  # will open a saved project before linking the Jupyter session

#### Execute simulation and verify convergence

Create and activate an eigenmode design called "TransmonQubit".

In [1]:
hfss.activate_ansys_design("TransmonQubit", 'eigenmode')  # use new_ansys_design() to force creation of a blank design

NameError: name 'hfss' is not defined

Render the single qubit in Metal, called Q1, to "TransmonQubit" design in Ansys.

In [ ]:
hfss.render_design(['Q1'], [])
hfss.save_screenshot()

Set the convergence parameters and junction properties in the Ansys design. <br>
Then run the analysis and plot the convergence.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')

setup.analyze()

To plot the results you can use the `plot_convergences()` method from the `eig_qb.sim` object. The method will read the data from the variables local to the `eig_qb.sim` object, so we first need to assign the simulation results to these two variables. let's do both (assignment and plotting) in the next cell.

In [ ]:
eig_qb.sim.convergence_t, eig_qb.sim.convergence_f, _ = hfss.get_convergences()
eig_qb.sim.plot_convergences()

#### Plot the EM field for inspection
Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot to prepare for the next phase.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

### EPR Analysis
In the `suggested` (tutorial 4.2) flow, we would now prepare the setup using `eig_qb.setup` and run the analysis with `eig_qb.run_epr()`. Notice that this method requires previous set of the `eig_qb` variables `convergence_t` and `convergence_f` like we did a thee cells earlier.

However we here exemplify the advanced approach, which is Ansys-specific since it uses the pyEPR module methods directly.
#### Setup
Identify the non-linear (Josephson) junctions in the model. You will need to list the junctions in the epr setup.

In this case there's only one junction, namely 'jj'. Let's see what we need to change in the default setup.

In [ ]:
pinfo = hfss.pinfo
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify

#### Execute the energy distribution analysis

Execute microwave analysis on eigenmode solutions.

In [ ]:
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag    = {ℰ_mag}
""")

#### Run the EPR analysis

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys session

In [ ]:
eig_qb.sim.close()

# 3. Analyze the transmon using the LOM method

In this section we will use a semi-manual (advanced) analysis flow. Please refer to tutorial 4.1 for the `suggested` method. As illustrated, the methods are equivalent, but the advanced method allows you to directly override some renderer-specific settings.

### Capacitance matrix extraction
#### Setup
Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation.

In [9]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

For the capacitive simulation portion, you can either:
1. Use the `c1` user-friendly methods (see tutorial 4.1)
2. Control directly the simulation tool from the tool's GUI (outside metal - see specific vendor instructions)
3. Use the renderer methods
In this section we show the advanced method (method 3).

The renderer can be reached from the analysis class. Let's give it a shorter alias.

In [10]:
q3d = c1.sim.renderer

Now we connect to the simulation tool, similarly to what we have done for the eigenmode analysis.

In [11]:
q3d.start()

INFO 04:57PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:57PM [load_ansys_project]: 	Opened Ansys App
INFO 04:57PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 04:57PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Berkelium/Documents/Ansoft/
	Project:   Project6
INFO 04:57PM [connect_design]: No active design found (or error getting active design).
INFO 04:57PM [connect]: 	 Connected to project "Project6". No design detected


True

If the simulator is already open, the line above will simply connect to the open session, project and design.

#### Execute simulation and verify convergence

Create and activate a q3d design called "TransmonQubit_q3d".

In [12]:
q3d.activate_ansys_design("TransmonQubit_q3d", 'capacitive')  # use new_ansys_design() to force creation of a blank design

04:57PM 43s WARNING [activate_ansys_design]: The design_name=TransmonQubit_q3d was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:57PM [connect_design]: 	Opened active design
	Design:    TransmonQubit_q3d [Solution type: Q3D]
WARNING 04:57PM [connect_setup]: 	No design setup detected.
WARNING 04:57PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:57PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


Next, we render the qubit to Ansys Q3D for analysis. We set the readout pin of the qubit in the 'open' termination list of the render so its capacitance is properly simulated.

In [13]:
q3d.render_design(['Q1'], [('Q1','readout')])

Execute the capacitance extraction and verify converengence. This cell analyzes the default setup.

In [15]:
q3d.analyze_setup("Setup")

INFO 04:58PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:58PM [analyze]: Analyzing setup Setup


This simulation had 4 nets, the two charge islands of the floating transmon, the readout coupler, and the ground, resulting in a 4x4 capacitance matrix. Output is of type DataFrame.

In [16]:
c1.sim.capacitance_matrix, c1.sim.units = q3d.get_capacitance_matrix()
c1.sim.capacitance_all_passes, _ = q3d.get_capacitance_all_passes()
c1.sim.capacitance_matrix

INFO 04:58PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmp0rtylz47.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:58PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmpjw_btd3i.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:58PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmppfgne2qo.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 04:58PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmp75neuf8v.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 04:58PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmp9zw4avyg.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 4, F

,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
ground_main_plane,177.78538,-44.74611,-38.34722,-37.03041
pad_bot_Q1,-44.74611,82.84141,-32.48515,-2.30760
pad_top_Q1,-38.34722,-32.48515,93.38943,-19.67329
readout_connector_pad_Q1,-37.03041,-2.30760,-19.67329,60.14998


### LOM Analysis
Now we provide the junction lumped element values, and complete the analysis by plotting the convergence. This is the same steps used in the `suggested` flow from tutorial 4.1.

In [17]:
c1.setup.junctions=Dict(Lj=12.31, Cj=2)
c1.setup.freq_readout = 7.0
c1.setup.freq_bus = []

c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.492369 [GHz]
EC 320.340887 [MHz]
EJ 13.273404 [GHz]
alpha -374.552102 [MHz]
dispersion 58.767574 [KHz]
Lq 12.305036 [nH]
Cq 60.467548 [fF]
T1 47.110126 [us]

**Coupling Properties**

tCqbus1 -7.535751 [fF]
gbus1_in_MHz -119.111599 [MHz]
χ_bus1 -3.815566 [MHz]
1/T1bus1 3378.359561 [Hz]
T1bus1 47.110126 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.848645,366.675723,13.273404,-435.120535,183.838223,[-120.91915969031888],[-7.045838787793927],7.045839,120.919160
2,5.800484,360.198673,13.273404,-426.554207,158.894188,[-112.57027192270645],[-5.610759520303457],5.610760,112.570272
3,5.728978,350.706154,13.273404,-414.05886,127.373567,[-114.4721344200509],[-5.13583772345763],5.135838,114.472134
4,5.645726,339.839946,13.273404,-399.840701,97.757395,[-113.75236108111946],[-4.422671613573631],4.422672,113.752361
5,5.574615,330.715418,13.273404,-387.971176,77.483483,[-115.42485160814473],[-4.067027529551065],4.067028,115.424852
6,5.552051,327.850119,13.273404,-384.256913,71.883362,[-116.90656488361248],[-4.027920633489713],4.027921,116.906565
7,5.522295,324.093652,13.273404,-379.396816,65.050231,[-118.53068129631139],[-3.9548493935624265],3.954849,118.530681
8,5.508895,322.410099,13.273404,-377.222089,62.166575,[-118.93049750280109],[-3.9007913880709433],3.900791,118.930498
9,5.492369,320.340887,13.273404,-374.552102,58.767574,[-119.11159900747404],[-3.8155660691693987],3.815566,119.111599


And plot the convergence.

In [18]:
c1.plot_convergence();
c1.plot_convergence_chi()

  self._hfss_variables[variation] = pd.Series(

INFO 04:59PM [hfss_report_full_convergence]: Creating report for variation 0


Design "TransmonQubit_q3d" info:
	# eigenmodes    0
	# variations    1


Release the simulator and close the analysis.

In [ ]:
c1.sim.close()

(optional) close the GUI.

In [ ]:
# gui.main_window.close()